In [1]:
#Imports
import pandas
import numpy as np
df = pandas.read_csv("training.csv", header=0, sep=",", encoding='Latin-1')
df.head()
df_test = pandas.read_csv("testing.csv", header=0, sep=",", encoding='Latin-1')
df_test.head()
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

In [15]:
df = df[['ItemID','Sentiment', 'SentimentText']]
df.head()
#df.shape

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [4]:
df_test = df_test[['ItemID','SentimentText']]
df_test.head()
#df_test.shape

,ItemID,SentimentText
0,1,is so sad for my APL frie...
1,2,I missed the New Moon trail...
2,3,omg its already 7:30 :O
3,4,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,i think mi bf is cheating on me!!! ...


In [5]:
#Text Normalization
import re

def norm1(row):
    return re.sub('[^a-zA-Z ]','', row['SentimentText'])
def norm2(row):
    return re.sub(r'\s+', ' ',  row['norm']) 
    
df['norm'] = df_test.apply(norm1, axis=1)
df_test['norm'] = df_test.apply(norm1, axis=1)
df['norm'] = df_test.apply(norm2, axis=1)
df_test['norm'] = df_test.apply(norm2, axis=1)
df.head()

,ItemID,Sentiment,SentimentText,norm
0,1,0,is so sad for my APL frie...,is so sad for my APL friend
1,2,0,I missed the New Moon trail...,I missed the New Moon trailer
2,3,1,omg its already 7:30 :O,omg its already O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...,Omgaga Im sooo im gunna CRy Ive been at this ...
4,5,0,i think mi bf is cheating on me!!! ...,i think mi bf is cheating on me TT


In [6]:
#Text conversion
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
norm = df['norm']
norm1 = df_test['norm']
cv.fit(norm)
X = cv.transform(norm)
X_pred = cv.transform(norm1)
y = df[['Sentiment']]
X.shape
X_pred.shape

(299989, 113701)

In [7]:
#Traning dataset split
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

/home/mcslk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
#Neural Network Definition
from keras import models
from keras.layers import Dense, Dropout

model = models.Sequential()
model.add(Dense(2048, activation='relu', input_shape=(113701,)))
model.add(Dropout(0.3))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2048)              232861696 
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 1026      
Total para

In [16]:
#Compilation
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [10]:
#Model Training
results = model.fit(
 X_train, y_train,
 epochs = 2,
 batch_size = 512,
 validation_data = (X_test, y_test)
)

Train on 74991 samples, validate on 24998 samples
Epoch 1/2
74991/74991 [==============================] - 1009s 13ms/step - loss: 0.5082 - acc: 0.7476 - val_loss: 0.4718 - val_acc: 0.7739
Epoch 2/2
74991/74991 [==============================] - 988s 13ms/step - loss: 0.2197 - acc: 0.9130 - val_loss: 0.5941 - val_acc: 0.7489


In [11]:
#Model Validation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5941298660855072
Test accuracy: 0.7488999119977282


In [12]:
#Prediction on Test Set
prediction1 = model.predict(X_pred, batch_size=512, verbose=0, steps=None)

In [13]:
#Save DF to File
prediction_df = pandas.DataFrame(prediction1)
prediction_df.head()
prediction_df.to_csv('result_kerras_final.csv', index=False)

,0,1
0,0.986835,0.013165
1,0.990366,0.009634
2,0.829672,0.170328
3,0.999668,0.000332
4,0.999974,0.000027
